In [ ]:
## Importing Libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

Loading and Preprocessing the Dataset (with the batch_size of 16)

In [ ]:
#normalize the dataset so that each channel has zero mean and unitary standard deviation.
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 16
# loading datasets with dataloaders
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
# Classes (10)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Function for training and testing the models

In [ ]:
# Develope a train and test mechanism 
def train_and_test(model, optimizer, loss_fn, train_loader, test_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            # Reset the gradient
            optimizer.zero_grad()
            # Allocate inputs and targets to batch for training
            inputs, targets = batch
            # Selecting the processor
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            # Calculate loss
            loss = loss_fn(output, targets)
            # Back pass
            loss.backward()
            optimizer.step()
            # Calculation of loss
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        # Evaluating the model on test
        model.eval()
        num_correct = 0 
        num_examples = 0
        # loop within the batch
        for batch in test_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            # Evaluating the performance of the model
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(test_loader.dataset)
        # Accuracy
        accuracy = num_correct/num_examples
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, accuracy))



## A Simple Neural Network

This is a Simple Feed Forward neural network made without the proposed improvements

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Input layer, taking in the input values, in our case, are 32*32*3 and an arbitrary output to the first hidden layer
        self.fc1 = nn.Linear(32*32*3, 84) 
        self.fc2 = nn.Linear(84, 50) # 
        self.fc3 = nn.Linear(50,10) # 84 to 10
    def forward(self, x):
        
        # Convert to 1D vector
        x = x.view(-1, 32*32*3)
        # layers of ReLu
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiation
simplenet = SimpleNet()

# Choosing Optimizer with learning rate
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

# Preferring
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)


SimpleNet(
  (fc1): Linear(in_features=3072, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
)

In [ ]:
# train and test
train_and_test(simplenet, optimizer,torch.nn.CrossEntropyLoss(), trainloader, testloader, epochs=10, device=device)

Epoch: 1, Training Loss: 1.66, Validation Loss: 1.52, accuracy = 0.46
Epoch: 2, Training Loss: 1.48, Validation Loss: 1.48, accuracy = 0.48
Epoch: 3, Training Loss: 1.40, Validation Loss: 1.44, accuracy = 0.49
Epoch: 4, Training Loss: 1.34, Validation Loss: 1.43, accuracy = 0.50
Epoch: 5, Training Loss: 1.30, Validation Loss: 1.44, accuracy = 0.50
Epoch: 6, Training Loss: 1.26, Validation Loss: 1.41, accuracy = 0.51
Epoch: 7, Training Loss: 1.23, Validation Loss: 1.44, accuracy = 0.50
Epoch: 8, Training Loss: 1.20, Validation Loss: 1.45, accuracy = 0.51
Epoch: 9, Training Loss: 1.17, Validation Loss: 1.45, accuracy = 0.51
Epoch: 10, Training Loss: 1.14, Validation Loss: 1.43, accuracy = 0.51


## Neural Network with enhancements like Convolutional Layers, DropOut, and MaxPooling

In [ ]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # input / output channels and kernel
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
cnnnet = CNNNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)
cnnnet.to(device)
train_and_test(cnnnet, optimizer, criterion, trainloader, testloader, epochs=10, device=device)


Epoch: 1, Training Loss: 1.55, Validation Loss: 1.35, accuracy = 0.51
Epoch: 2, Training Loss: 1.28, Validation Loss: 1.22, accuracy = 0.56
Epoch: 3, Training Loss: 1.16, Validation Loss: 1.15, accuracy = 0.60
Epoch: 4, Training Loss: 1.08, Validation Loss: 1.16, accuracy = 0.59
Epoch: 5, Training Loss: 1.02, Validation Loss: 1.15, accuracy = 0.61
Epoch: 6, Training Loss: 0.97, Validation Loss: 1.11, accuracy = 0.62
Epoch: 7, Training Loss: 0.92, Validation Loss: 1.09, accuracy = 0.63
Epoch: 8, Training Loss: 0.88, Validation Loss: 1.05, accuracy = 0.64
Epoch: 9, Training Loss: 0.85, Validation Loss: 1.08, accuracy = 0.64
Epoch: 10, Training Loss: 0.82, Validation Loss: 1.10, accuracy = 0.63


In [ ]:
class CNNNet1(nn.Module):

    def __init__(self, num_classes=10):
        super(CNNNet1, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(32, 128, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
     
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
# Intantiation
cnnnet1 = CNNNet1()


In [ ]:
cnnnet1.to(device)

# Optimizer with learning rate
optimizer = optim.Adam(cnnnet1.parameters(), lr=0.001) 

# Loss Criterion
criterion = nn.CrossEntropyLoss()


In [ ]:
train_and_test(cnnnet1, optimizer, criterion, trainloader, testloader, epochs=10, device=device)


Epoch: 1, Training Loss: 1.63, Validation Loss: 1.39, accuracy = 0.50
Epoch: 2, Training Loss: 1.36, Validation Loss: 1.28, accuracy = 0.54
Epoch: 3, Training Loss: 1.24, Validation Loss: 1.28, accuracy = 0.55
Epoch: 4, Training Loss: 1.17, Validation Loss: 1.19, accuracy = 0.58
Epoch: 5, Training Loss: 1.11, Validation Loss: 1.16, accuracy = 0.59
Epoch: 6, Training Loss: 1.06, Validation Loss: 1.20, accuracy = 0.58
Epoch: 7, Training Loss: 1.02, Validation Loss: 1.12, accuracy = 0.61
Epoch: 8, Training Loss: 0.98, Validation Loss: 1.13, accuracy = 0.61
Epoch: 9, Training Loss: 0.95, Validation Loss: 1.13, accuracy = 0.61
Epoch: 10, Training Loss: 0.91, Validation Loss: 1.14, accuracy = 0.61


Lets increase the complexity of the model by resizing the input, adding some extra hidden layers as well as Adaptive Pooling

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64,64)), # resize the images from 32*32 to 64*64
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 16

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class CNNNet2(nn.Module):

    def __init__(self, num_classes=10):
        super(CNNNet2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((3, 3))
        self.classifier = nn.Sequential(
            nn.Dropout(0.10),
            nn.Linear(256 * 3 * 3, 1028),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(1028, 1028),
            nn.ReLU(),
            nn.Linear(1028, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
# Intantiation
cnnnet2 = CNNNet2()


In [ ]:
cnnnet2.to(device)

# Optimizer with learning rate
optimizer = optim.Adam(cnnnet2.parameters(), lr=0.001) 

In [ ]:
train_and_test(cnnnet2, optimizer,torch.nn.CrossEntropyLoss(), trainloader, testloader, epochs=10, device=device)


Epoch: 1, Training Loss: 1.91, Validation Loss: 1.70, accuracy = 0.34
Epoch: 2, Training Loss: 1.63, Validation Loss: 1.56, accuracy = 0.42
Epoch: 3, Training Loss: 1.51, Validation Loss: 1.47, accuracy = 0.47
Epoch: 4, Training Loss: 1.43, Validation Loss: 1.39, accuracy = 0.49
Epoch: 5, Training Loss: 1.38, Validation Loss: 1.41, accuracy = 0.51
Epoch: 6, Training Loss: 1.34, Validation Loss: 1.35, accuracy = 0.52
Epoch: 7, Training Loss: 1.31, Validation Loss: 1.31, accuracy = 0.53
Epoch: 8, Training Loss: 1.28, Validation Loss: 1.30, accuracy = 0.54
Epoch: 9, Training Loss: 1.26, Validation Loss: 1.25, accuracy = 0.56
Epoch: 10, Training Loss: 1.24, Validation Loss: 1.36, accuracy = 0.52


## Summary
- A simple neural net could attain a mediocre Accuracy
- Increasing batchsize resulted in better accuracy. So does an increase in Drop out.
- Increasing the learning rate does increase the accuracy to some extent, but I doubt that the model is rather getting stuck in the incorrect minima

## Proposals:
- The scheduled decrease of learning rate of the epochs (for example, after each 10th)
- Using the Optmizer (SGD) with momentum less than 1
- Increament in the batchsize
- Batch Normalization
- Data Augmentation (gradual) to the training set
